In [1]:
#webcam2
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(img):
    global _grade
    # Extract features
    features = extract_features(img)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)
        # print(grade)
        _grade = grade
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

def main():
    # Set desired camera resolution
    desired_width = 1280
    desired_height = 720
    # Open the default camera (usually the built-in webcam)
    cap = cv2.VideoCapture(2)
    # cap.set(cv2.CAP_PROP_SETTINGS, 1)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    # Create a background subtractor object
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Check if the frame was captured successfully
        if not ret:
            print("Error: Failed to capture frame.")
            break

         # Apply background subtraction
        fg_mask = bg_subtractor.apply(frame)
        fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)
    
        # Convert the image to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (31, 31), 0)
    
        # Perform adaptive thresholding to separate foreground from background
        _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
        # Invert the thresholded image
        thresholded = cv2.bitwise_not(thresholded)
    
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the largest contour (if any)
        largest_contour = max(contours, key=cv2.contourArea, default=None)

        # Draw bounding box around the largest contour (if any)
        if largest_contour is not None:
            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Zoom in to a specific area inside the bounding box
            zoom_factor = 3  # Adjust the zoom factor as needed
            center_x = x + w // 2
            center_y = y + h // 2
            zoom_size = min(w, h) // zoom_factor
            x_zoom = max(0, center_x - zoom_size // 2)
            y_zoom = max(0, center_y - zoom_size // 2)
            zoomed_image = frame[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

            try:
                # Resize the zoomed image to 100x100 pixels
                zoomed_resized = cv2.resize(zoomed_image, (400, 400))
                cv2.imshow('Zoomed Bounding Box', zoomed_resized)
                zoomed_resized = cv2.resize(zoomed_image, (32, 32))
                classify_image(zoomed_resized)
                # Draw the outline of the text on the zoomed image
                outline_thickness = 2
                cv2.putText(frame, "Predicted Grade: {}".format(_grade), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)
                
                # Draw the text on the zoomed image
                cv2.putText(frame, "Predicted Grade: {}".format(_grade), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                # print(f"grade = {_grade}")
            except:
                 pass

        # Display the frame with bounding box (if any)
        cv2.imshow('Tuna Grade Prediction Using Knn', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close the windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

#webcam2

In [2]:
%%time
#input image
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

     # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))
    
    # Extract features
    features = extract_features(zoomed_resized)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image,(700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)
        
        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)
    
        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to open file explorer for picking an image
def open_file_explorer():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Open file explorer and return the selected file path
    file_path = filedialog.askopenfilename()

    if file_path:
        # If a file is selected, classify the image
        predicted_grade = classify_image(file_path)
        print("Predicted Grade:", predicted_grade)
    else:
        print("No file selected.")

# Call the function to open file explorer
open_file_explorer()


Predicted Grade: B
CPU times: total: 1.97 s
Wall time: 6.13 s


In [10]:

%%time
#perf evval
#input image
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math
import psutil
import time
import os

# Function to log performance metrics (CPU and RAM usage)
def log_performance_metrics(stage):
    process = psutil.Process(os.getpid())
    cpu_usage = psutil.cpu_percent(interval=1)
    ram_usage = process.memory_info().rss / 1024 / 1024  # Convert bytes to MB
    print(f"{stage} - CPU Usage: {cpu_usage}% | RAM Usage: {ram_usage:.2f} MB")

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

     # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (31, 31), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))
    
    # Extract features
    features = extract_features(zoomed_resized)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image,(700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)
        
        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
        # Display the frame with bounding box
        cv2.imshow('Tuna Grade Classification Knn', image)

        zoomed_resized = cv2.resize(zoomed_resized, (500, 500))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)
    
        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to open file explorer for picking an image
def open_file_explorer():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Open file explorer and return the selected file path
    file_path = filedialog.askopenfilename()

     # Measure and log performance before processing
    log_performance_metrics("Before Processing")

    # Start timing
    start_time = time.time()

    if file_path:
        # If a file is selected, classify the image
        predicted_grade = classify_image(file_path)
        print("Predicted Grade:", predicted_grade)

        # End timing
        end_time = time.time()
        processing_time = end_time - start_time
        print(f"Processing Time: {processing_time:.2f} seconds")
        
        # Measure and log performance after processing
        log_performance_metrics("After Processing")
    else:
        print("No file selected.")

# Call the function to open file explorer
open_file_explorer()


Before Processing - CPU Usage: 3.9% | RAM Usage: 216.45 MB
Predicted Grade: A
Processing Time: 49.96 seconds
After Processing - CPU Usage: 9.2% | RAM Usage: 1127.30 MB
CPU times: total: 3.64 s
Wall time: 55.1 s
